In [ ]:
import pylab as py
import numpy as np
import random as rn
import time
import scipy.special.cython_special
%run Jackknife.ipynb
pi = np.pi

In [ ]:
def dh4(f,x):
    '''
    4th order numerical derivative.
    This is just in case.
    '''
    f_prima = np.zeros(len(f))
    h = (x[1] - x[0]) 
    h_12 = 12*h
    for i in range(2,len(f)-2):
            f_prima[i] = ( -f[i+2] + 8 * (f[i+1]-f[i-1]) + f[i-2])/(h_12) 
    c0,c1,c2,c3,c4 = -25/12, 4, -3, 4/3, -0.25
    f_prima[0] = (c0*f[0] + c1*f[1] + c2*f[2] + c3*f[3]+c4*f[4])/h
    f_prima[1] = (c0*f[1] + c1*f[2] + c2*f[3] + c3*f[4]+c4*f[5])/h
    f_prima[-1] = -(c0*f[-1] + c1*f[-2] + c2*f[-3] + c3*f[-4]+c4*f[-5])/h
    f_prima[-2] = -(c0*f[-2] + c1*f[-3] + c2*f[-4] + c3*f[-5]+c4*f[-6])/h                
    return f_prima

def BesselITable(beta,max_nu):
    '''
    Generates a table (list) with the values of the modified Bessel function
    of the first kind.
    ''' 
    Inu = [] #Bessel function
    Inu1 = [] #First derivative
    Inu2 = [] #Second derivative
    for i in range(max_nu+1):
        Inu.append(scipy.special.iv(i, beta, out=None))
        Inu1.append(scipy.special.ivp(i, beta, n=1))
        Inu2.append(scipy.special.ivp(i, beta, n=2))
    return Inu, Inu1, Inu2

def BesselITableCaller(beta,Jij,Inu,Inu1,Inu2):
    '''
    This function decides whether to call a function from the table 
    or to generate a new one which is not in the table.
    '''
    Jij = int(Jij)
    max_nu = len(Inu)-1  
    if abs(Jij) <= max_nu:
        Iv, Iv1, Iv2 = Inu[abs(Jij)], Inu1[abs(Jij)], Inu2[abs(Jij)]
    else:
        Iv, Iv1, Iv2 = scipy.special.iv(Jij, beta, out=None), scipy.special.ivp(Jij, beta, n=1), scipy.special.ivp(Jij, beta, n=2)
    return Iv, Iv1, Iv2

def Energy(L,beta,FluxX, FluxY,Inu,Inu1,Inu2):
    '''
    Computes E and E² for each measurement.
    '''
    En = 0 #Energy
    En2 = 0  #Squared energy
    for i in range(L): 
        for j in range(L):
            IvX, Iv1X, Iv2X = BesselITableCaller(beta,FluxX[i,j],Inu,Inu1,Inu2)
            IvY, Iv1Y, Iv2Y = BesselITableCaller(beta,FluxY[i,j],Inu,Inu1,Inu2)
            En += Iv1X/IvX + Iv1Y/IvY
            En2 += Iv2X/IvX + Iv2Y/IvY - (Iv1X/IvX)**2 - (Iv1Y/IvY)**2           
    En2 += En**2
    return -En, En2
            
def AccRatio(mu,beta,L,Masha,FluxX, FluxY):
    '''
    This function computes the acceptance ratio based on the flux between
    two neighbouring sites.
    '''
    if mu == 0:
        FluxIni = FluxX[Masha[0],Masha[1]]
    elif mu == 1:
        FluxIni = -FluxX[Masha[0],(Masha[1]-1)%L]
    elif mu == 2:
        FluxIni = - FluxY[(Masha[0]-1)%L,Masha[1]]
    elif mu == 3:
        FluxIni = FluxY[Masha[0],Masha[1]]
    FluxFin = FluxIni + 1    
    R = scipy.special.iv(FluxFin, beta, out=None)/scipy.special.iv(FluxIni, beta, out=None)          
    return R
            
def WA_sweep(L,beta,FluxX, FluxY, Masha):
    '''
    Performs one sweep with the worm algorithm, or better to say moves the head
    of the worm.
    '''
    mu = rn.randint(0, 3) #0 --> +x direction, 1 --> -x direction, 2 --> +y direction, 3 --> -y direction
    R = AccRatio(mu,beta,L,Masha,FluxX, FluxY)
    r = rn.random() #random number between zero and one  
    if r < R:
        if mu == 0: 
            FluxX[Masha[0], Masha[1]] += 1   #Create bond from Masha to the right site -->
            Masha[1] = (Masha[1]+1) % L #move masha one site to the right           
        elif mu == 1: 
            FluxX[Masha[0],(Masha[1]-1) % L] -= 1  #Create bond from Masha to the left site <--
            Masha[1] = (Masha[1]-1) % L #move masha one site to the left    
        elif mu == 2: 
            FluxY[(Masha[0]-1)%L, Masha[1]] -= 1 #Create bond from Masha to the site above         
            Masha[0] = (Masha[0]-1) % L #move masha up   
        elif mu == 3: 
            FluxY[Masha[0], Masha[1]] += 1 #Create bond from Masha to the site below
            Masha[0] = (Masha[0]+1) % L #move masha dowm
    return Masha, FluxX, FluxY

def WA_XY2d(beta,L,Ntherm,Nmeas,Nsteps):
    '''
    Worm algorithm for one value of beta.
    '''
    Inu, Inu1, Inu2 = BesselITable(beta,5) #This computes the modified bessel functions and its derivatives up to nu = 5
    Ira = [rn.randint(0, L-1), rn.randint(0,L-1)] #randint(a,b) generates a random number r\in[a,b] (closed interval)
    Masha = [Ira[0],Ira[1]]
    FluxX, FluxY = np.zeros((L,L)), np.zeros((L,L))
    Chi = []
    En = []
    En2 = []
    G = np.zeros(L**2)
    #----Thermalization steps----#
    for i in range(Ntherm):
        if Masha == Ira: 
            Ira = [rn.randint(0, L-1), rn.randint(0,L-1)]
            Masha = [Ira[0],Ira[1]]
        Masha, FluxX, FluxY = WA_sweep(L,beta,FluxX, FluxY,Masha)
    #---------------------------#
    count = 0
    #----Measurements----#
    while count<Nmeas:
        SiteIra = Ira[1] + Ira[0]*L
        SiteMasha = Masha[1] + Masha[0]*L
        G[abs(SiteIra-SiteMasha)] += 1
        if Masha == Ira:
            E, E2 = Energy(L,beta,FluxX,FluxY,Inu,Inu1,Inu2) #E and E²
            En.append(E)
            En2.append(E2)
            Chi.append(np.sum(G)/G[0])
            Ira = [rn.randint(0, L-1), rn.randint(0,L-1)] #move the head of the worm and begin somewhere else.
            Masha = [Ira[0],Ira[1]] 
            count += 1 #Update the measurements counter.
        Masha, FluxX, FluxY = WA_sweep(L,beta,FluxX, FluxY,Masha)
    #----Decorrelation steps----#
        for j in range(Nsteps): 
            SiteIra = Ira[1] + Ira[0]*L
            SiteMasha = Masha[1] + Masha[0]*L
            G[abs(SiteIra-SiteMasha)] += 1
            if Masha == Ira:
                Ira = [rn.randint(0, L-1), rn.randint(0,L-1)]
                Masha = [Ira[0],Ira[1]]
            Masha, FluxX, FluxY = WA_sweep(L,beta,FluxX, FluxY,Masha)
            
    En = np.array(En)
    E = np.mean(En) #<E>
    dE = Jerr(En)  #d<E>
    En2 = np.array(En2) 
    E2 =np.mean(En2) #<E²>
    dE2 = Jerr(En2) #d<E²>
    
    Cv = beta**2 * (E2-E**2)  #Cv*L² = beta²*(<E²>-<E>²)
    dCv = abs(beta**2 * (dE2 + 2*E*dE )) #dCv  
    
    Chi = np.array(Chi) 
    chi = np.mean(Chi) #Chi = <M>
    dchi = Jerr(Chi) 
    return chi, dchi, E, dE, Cv, dCv

def ChidChiResults(L,Ntherm,Nmeas,Nsteps,beta_in,beta_fin,beta_step):
    '''
    Performs simulations for several values of beta.
    '''
    beta = np.linspace(beta_in,beta_fin,beta_step)
    Chi, dChi = [], []
    En, dEn = [], []
    Cv, dCv = [], []
    for i in range(len(beta)):
        chi, dchi, E, dE, cv, dcv = WA_XY2d(beta[i],L,Ntherm,Nmeas,Nsteps)
        Chi.append(chi)
        dChi.append(dchi)
        En.append(E)
        dEn.append(dE)
        Cv.append(cv)
        dCv.append(dcv)
        print('i',i,'beta',beta[i],'T',1/beta[i])
        print('chi',chi,'+-',dchi)
        print('Energy',E,'+-',dE)
        print('Cv',cv,'+-',dcv)
        print('---------------------------')
    dChi = np.array(dChi)
    Chi = np.array(Chi)
    En = np.array(En)
    dEn = np.array(dEn)
    Cv = np.array(Cv)
    dCv = np.array(dCv)
    return Chi, dChi, En, dEn, Cv, dCv

In [ ]:
Ntherm, Nmeas, Nsteps = 1000, 1000, 10
beta_ini, beta_fin, beta_step = 0.1, 1.4, 30
beta = np.linspace(beta_ini,beta_fin,beta_step)

L = 8
print('L={0}'.format(L))
Chi8, dChi8, E8, dE8, Cv8, dCv8 = ChidChiResults(L,Ntherm,Nmeas,Nsteps,beta_ini,beta_fin,beta_step)

L = 10
print('L={0}'.format(L))
Chi10, dChi10, E10, dE10, Cv10, dCv10  = ChidChiResults(L,Ntherm,Nmeas,Nsteps,beta_ini,beta_fin,beta_step)

L = 16
print('L={0}'.format(L))
Chi16, dChi16, E16, dE16, Cv16, dCv16  = ChidChiResults(L,Ntherm,Nmeas,Nsteps,beta_ini,beta_fin,beta_step)

L = 24
print('L={0}'.format(L))
Chi24, dChi24, E24, dE24, Cv24, dCv24  = ChidChiResults(L,Ntherm,Nmeas,Nsteps,beta_ini,beta_fin,beta_step)

In [ ]:
fig = py.figure(dpi=150)
py.errorbar(beta,Chi8/8**2,yerr=dChi8/8**2,fmt='o',markersize='2',\
elinewidth=0.5,solid_capstyle='projecting', capsize=1.5,label=r'$L=${0}'.format(8))
py.errorbar(beta,Chi10/10**2,yerr=dChi10/10**2,fmt='o',markersize='2',\
elinewidth=0.5,solid_capstyle='projecting', capsize=1.5,label=r'$L=${0}'.format(10))
py.errorbar(beta,Chi16/16**2,yerr=dChi16/16**2,fmt='o',markersize='2',\
elinewidth=0.5,solid_capstyle='projecting', capsize=1.5,label=r'$L=${0}'.format(16))
py.errorbar(beta,Chi24/24**2,yerr=dChi24/24**2,fmt='o',markersize='2',\
elinewidth=0.5,solid_capstyle='projecting', capsize=1.5,label=r'$L=${0}'.format(24))
py.title('Nmeas={0}, Nstep={1}'.format(Nmeas,Nsteps))
py.xlabel(r'$\beta$')
py.ylabel(r'$\frac{\langle M \rangle}{L^2}$',rotation=0)
py.legend()
py.grid()
py.tight_layout()
fig.savefig('2DXYMagDenAllL.pdf')
py.show()

fig = py.figure(dpi=150)
py.errorbar(beta,E8/8**2,yerr=dE8/8**2,fmt='o',markersize='2',\
elinewidth=0.5,solid_capstyle='projecting', capsize=1.5,label=r'$L=${0}'.format(8))
py.errorbar(beta,E10/10**2,yerr=dE10/10**2,fmt='o',markersize='2',\
elinewidth=0.5,solid_capstyle='projecting', capsize=1.5,label=r'$L=${0}'.format(10))
py.errorbar(beta,E16/16**2,yerr=dE16/16**2,fmt='o',markersize='2',\
elinewidth=0.5,solid_capstyle='projecting', capsize=1.5,label=r'$L=${0}'.format(16))
py.errorbar(beta,E24/24**2,yerr=dE24/24**2,fmt='o',markersize='2',\
elinewidth=0.5,solid_capstyle='projecting', capsize=1.5,label=r'$L=${0}'.format(24))
py.title('Nmeas={0}, Nstep={1}'.format(Nmeas,Nsteps))
py.xlabel(r'$\beta$')
py.ylabel(r'$\frac{\langle E \rangle}{L^2}$',rotation=0)
py.legend()
py.grid()
py.tight_layout()
fig.savefig('2DXYEnAllL.pdf')
py.show()

fig = py.figure(dpi=150)
py.errorbar(beta,Cv8/8**2,yerr=dCv8/8**2,fmt='o',markersize='2',\
elinewidth=0.5,solid_capstyle='projecting', capsize=1.5,label=r'$L=${0}'.format(8))
py.errorbar(beta,Cv10/10**2,yerr=dCv10/10**2,fmt='o',markersize='2',\
elinewidth=0.5,solid_capstyle='projecting', capsize=1.5,label=r'$L=${0}'.format(10))
py.errorbar(beta,Cv16/16**2,yerr=dCv16/16**2,fmt='o',markersize='2',\
elinewidth=0.5,solid_capstyle='projecting', capsize=1.5,label=r'$L=${0}'.format(16))
py.errorbar(beta,Cv24/24**2,yerr=dCv24/24**2,fmt='o',markersize='2',\
elinewidth=0.5,solid_capstyle='projecting', capsize=1.5,label=r'$L=${0}'.format(24))
py.title('Nmeas={0}, Nstep={1}'.format(Nmeas,Nsteps))
py.xlabel(r'$\beta$')
py.ylabel(r'$C_v$',rotation=0)
py.legend()
py.grid()
py.tight_layout()
fig.savefig('2DXYCvAllL.pdf')
py.show()

# Debug and testing

This part is to debug or test the code.

In [ ]:
def TotalFlux(Coords,FluxX, FluxY):
    return (FluxX[Coords[0],Coords[1]] - FluxX[Coords[0],(Coords[1]-1)%L] + FluxY[Coords[0],Coords[1]] - FluxY[(Coords[0]-1)%L,Coords[1]])

def TotalFluxMatrix(L,FluxX,FluxY):
    Flux = np.zeros((L,L))
    for i in range(L):
        for j in range(L):
            Flux[i,j] = TotalFlux([i,j],FluxX,FluxY)
    return Flux

def BesselITable(beta,max_nu):
    Inu = [] #Bessel function
    Inu1 = [] #First derivative
    Inu2 = [] #Second derivative
    for i in range(max_nu+1):
        Inu.append(scipy.special.iv(i, beta, out=None))
        Inu1.append(scipy.special.ivp(i, beta, n=1))
        Inu2.append(scipy.special.ivp(i, beta, n=2))
    return Inu, Inu1, Inu2

def BesselITableCaller(beta,Jij,Inu,Inu1,Inu2):
    Jij = int(Jij)
    #Jij: FLux from i to j
    max_nu = len(Inu)-1  
    if abs(Jij) <= max_nu:
        Iv, Iv1, Iv2 = Inu[abs(Jij)], Inu1[abs(Jij)], Inu2[abs(Jij)]
    else:
        Iv, Iv1, Iv2 = scipy.special.iv(Jij, beta, out=None), scipy.special.ivp(Jij, beta, n=1), scipy.special.ivp(Jij, beta, n=2)
    return Iv, Iv1, Iv2

def Energy(L,beta,FluxX, FluxY,Inu,Inu1,Inu2):
    '''
    Computes the E and E² for each measurement
    '''
    En = 0 #Energy
    En2 = 0  #Squared energy
    for i in range(L): 
        for j in range(L):
            IvX, Iv1X, Iv2X = BesselITableCaller(beta,FluxX[i,j],Inu,Inu1,Inu2)
            IvY, Iv1Y, Iv2Y = BesselITableCaller(beta,FluxY[i,j],Inu,Inu1,Inu2)
            En += Iv1X/IvX + Iv1Y/IvY
            En2 += Iv2X/IvX + Iv2Y/IvY - (Iv1X/IvX)**2 - (Iv1Y/IvY)**2           
    En2 += En**2
    return -En, En2
            
def AccRatio(mu,beta,L,Masha,FluxX, FluxY):
    if mu == 0:
        FluxIni = FluxX[Masha[0],Masha[1]]
    elif mu == 1:
        FluxIni = -FluxX[Masha[0],(Masha[1]-1)%L]
    elif mu == 2:
        FluxIni = - FluxY[(Masha[0]-1)%L,Masha[1]]
    elif mu == 3:
        FluxIni = FluxY[Masha[0],Masha[1]]
    FluxFin = FluxIni + 1
        
    R = scipy.special.iv(FluxFin, beta, out=None)/scipy.special.iv(FluxIni, beta, out=None)          
    return R
            
def WA_sweep(L,beta,FluxX, FluxY, Masha):   
    mu = rn.randint(0, 3) #0 --> +x direction, 1 --> -x direction, 2 --> +y direction, 3 --> -y direction
    R = AccRatio(mu,beta,L,Masha,FluxX, FluxY)
    r = rn.random() #random number between zero and one  
    if r < R:
        if mu == 0: 
            FluxX[Masha[0], Masha[1]] += 1   #Create bond from Masha to the right site -->
            Masha[1] = (Masha[1]+1) % L #move masha one site to the right           
        elif mu == 1: 
            FluxX[Masha[0],(Masha[1]-1) % L] -= 1  #Create bond from Masha to the left site <--
            Masha[1] = (Masha[1]-1) % L #move masha one site to the left    
        elif mu == 2: 
            FluxY[(Masha[0]-1)%L, Masha[1]] -= 1 #Create bond from Masha to the site above         
            Masha[0] = (Masha[0]-1) % L #move masha up   
        elif mu == 3: 
            FluxY[Masha[0], Masha[1]] += 1 #Create bond from Masha to the site below
            Masha[0] = (Masha[0]+1) % L #move masha dowm
    #print('mu',mu)
    return Masha, FluxX, FluxY

def WA_XY2d(beta,L,Ntherm,Nmeas,Nsteps):
    Inu, Inu1, Inu2 = BesselITable(beta,5) #This computes the modified bessel functions and its derivatives up to nu = 5
    Ira = [rn.randint(0, L-1), rn.randint(0,L-1)] #randint(a,b) generates a random number r\in[a,b] (closed interval)
    Masha = [Ira[0],Ira[1]]
    FluxX, FluxY = np.zeros((L,L)), np.zeros((L,L))
    Chi = []
    En = []
    En2 = []
    G = np.zeros(L**2)
    #----Thermalization steps----#
    for i in range(Ntherm):
        if Masha == Ira: 
            Ira = [rn.randint(0, L-1), rn.randint(0,L-1)]
            Masha = [Ira[0],Ira[1]]
        Masha, FluxX, FluxY = WA_sweep(L,beta,FluxX, FluxY,Masha)
    #---------------------------#
    count = 0
    #----Measurements----#
    while count<Nmeas:
        SiteIra = Ira[1] + Ira[0]*L
        SiteMasha = Masha[1] + Masha[0]*L
        G[abs(SiteIra-SiteMasha)] += 1
        if Masha == Ira:
            E, E2 = Energy(L,beta,FluxX,FluxY,Inu,Inu1,Inu2)
            En.append(E)
            En2.append(E2)
            count += 1
            Chi.append( (np.sum(G)+L**2*G[0])/G[0] )
            Ira = [rn.randint(0, L-1), rn.randint(0,L-1)]
            Masha = [Ira[0],Ira[1]] 
        Masha, FluxX, FluxY = WA_sweep(L,beta,FluxX, FluxY,Masha)
    #----Decorrelation steps----#
        for j in range(Nsteps): 
            SiteIra = Ira[1] + Ira[0]*L
            SiteMasha = Masha[1] + Masha[0]*L
            G[abs(SiteIra-SiteMasha)] += 1
            if Masha == Ira:
                Ira = [rn.randint(0, L-1), rn.randint(0,L-1)]
                Masha = [Ira[0],Ira[1]]
            Masha, FluxX, FluxY = WA_sweep(L,beta,FluxX, FluxY,Masha)
    En = np.array(En)
    E = np.mean(En)
    dE = Jerr(En)
    En2 = np.array(En2)
    E2 =np.mean(En2)
    dE2 = Jerr(En2)
    
    Cv = beta**2 * (E2-E**2)
    dCv = beta**2 * (dE2 + 2*E*dE )  
    
    Chi = np.array(Chi)
    chi = np.mean(Chi)
    dchi = Jerr(Chi) 
    return chi, dchi, E, dE, Cv, dCv

def ChidChiResults(L,Ntherm,Nmeas,Nsteps,beta_in,beta_fin,beta_step):
    beta = np.linspace(beta_in,beta_fin,beta_step)
    Chi, dChi = [], []
    En, dEn = [], []
    Cv, dCv = [], []
    for i in range(len(beta)):
        chi, dchi, E, dE, cv, dcv = WA_XY2d(beta[i],L,Ntherm,Nmeas,Nsteps)
        Chi.append(chi)
        dChi.append(dchi)
        En.append(E)
        dEn.append(dE)
        Cv.append(cv)
        dCv.append(dcv)
        print('i',i,'beta',beta[i],'T',1/beta[i])
        print('chi',chi,'+-',dchi)
        print('Energy',E,'+-',dE)
        print('Cv',cv,'+-',dcv)
        print('---------------------------')
    dChi = np.array(dChi)
    Chi = np.array(Chi)
    En = np.array(En)
    dEn = np.array(dEn)
    Cv = np.array(Cv)
    dCv = np.array(dCv)
    return Chi, dChi, En, dEn, Cv, dCv

In [ ]:
Ntherm, Nmeas, Nsteps = 100000, 1000, 10
beta_ini, beta_fin, beta_step = 0.1, 1.4, 30
beta = np.linspace(beta_ini,beta_fin,beta_step)
L = 10
print('L={0}'.format(L))
Chi, dChi, E, dE, Cv, dCv = ChidChiResults(L,Ntherm,Nmeas,Nsteps,beta_ini,beta_fin,beta_step)

In [ ]:
beta = np.linspace(beta_ini,beta_fin,beta_step)
fig = py.figure(dpi=150)
py.errorbar(beta,Chi/L**2,yerr=dChi/L**2,fmt='o',markersize='2',\
elinewidth=0.5,solid_capstyle='projecting', capsize=1.5,label=r'$L=${0}'.format(L))
py.title('Nmeas={0}, Nstep={1}'.format(Nmeas,Nsteps))
py.xlabel(r'$\beta$')
py.ylabel(r'$\frac{\langle M \rangle}{L^2}$',rotation=0)
py.legend()
py.grid()
py.xlim([0,1.41])
#py.ylim([0,0.9])
py.tight_layout()
py.show()

fig = py.figure(dpi=150)
py.errorbar(beta,E/L**2,yerr=dE/L**2,fmt='o',markersize='2',\
elinewidth=0.5,solid_capstyle='projecting', capsize=1.5,label=r'$L=${0}'.format(L))
py.title('Nmeas={0}, Nstep={1}'.format(Nmeas,Nsteps))
py.xlabel(r'$\beta$')
py.ylabel(r'$\frac{\langle E\rangle}{L^2}$',rotation=0)
py.legend()
py.grid()
py.xlim([0,1.41])
#py.ylim([0,0.9])
py.tight_layout()
py.show()

fig = py.figure(dpi=150)
py.errorbar(beta,Cv/L**2,yerr=dCv/L**2,fmt='o',markersize='2',\
elinewidth=0.5,solid_capstyle='projecting', capsize=1.5,label=r'$L=${0}'.format(L))
py.title('Nmeas={0}, Nstep={1}'.format(Nmeas,Nsteps))
py.xlabel(r'$\beta$')
py.ylabel(r'$C_V$',rotation=0)
py.legend()
py.grid()
py.xlim([0,1.41])
py.ylim([-0.6,2.5])
py.tight_layout()
py.show()